In [1]:
#Import required libraries

try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
# Create Json pipeline that can later be configured to database

import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('cityresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [3]:
#Setup spider that cralws looking for all city links listed on EventBrite website

import logging

class CitySpider(scrapy.Spider):
    name = 'eventbrite_cities'
    start_urls = [
        'https://www.eventbrite.com/directory/sitemap/',
    ]
    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'cityresult.json'                        # Used for pipeline 2
    }

    def parse(self, response):
        for city in response.css('div.g-cell'):
            yield {
                'link': city.css('a::attr("href")').get(),
            }

In [4]:
#Run crawler

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(CitySpider)
process.start()

2019-08-04 12:16:11 [scrapy.utils.log] INFO: Scrapy 1.7.3 started (bot: scrapybot)
2019-08-04 12:16:11 [scrapy.utils.log] INFO: Versions: lxml 4.2.1.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 19.2.1, Python 3.6.5 |Anaconda custom (64-bit)| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)], pyOpenSSL 18.0.0 (OpenSSL 1.1.1b  26 Feb 2019), cryptography 2.4.2, Platform Windows-10-10.0.17134-SP0
2019-08-04 12:16:11 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'cityresult.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
